## Benchmark model

In this notebook a Simple Moving Average model will be created in order to have a benchmark for my DeepAR based model (to be tuned in the next notebook).

### Hyperparameters

DeepAR is the model of choice of this project.
This model expects input data to be already test-train splitted.
A big part of the model design has to be done looking close at data.
More specifically, defining these two hyperparameters about the data:
* Context length
* Prediction length

### Prediction length

This is the length of the time series future predictions in days. It will be conveniently set to 5 days (exactly a week of trading hours) because a shorter interval would be of little significance.
A longer interval could be interesting from an application point of view, but it can be challenging in terms of model performances.

### Context length

Context length can be either:
* designed on patterns or seasonality observed in the data, if any is present;
* chosen as a fixed value. This will be my choice, and it will be the same as the moving average window, in order to have a good reference metrics, applicable to both this model and the benchmark model.

To explore this second option, we will refer to what we've found during the EDA stage.

In [134]:
prediction_length = [10, 20, 50]
context_length = [10, 20, 50]

### Train, test and validation split

Time series will be all trimmed from `data_start` in `1.ExploratoryDataAnalysis.ipy`, according to DeepAR documentation, train time series should be set as the size of the entire time series less the prediction length.
Validation length will exactly equal to prediction length.

In [135]:
from data.data_utils import train_test_valid_split

#### IBM Stock train, test and validation split

In [136]:
df_ibm = df_ibm.loc[data_start:].copy()

In [137]:
df_ibm_train, df_ibm_test, df_ibm_valid = train_test_valid_split(df_ibm, prediction_length=prediction_length[1])
print(len(df_ibm), len(df_ibm_train), len(df_ibm_test), len(df_ibm_valid))

4172 4132 4152 20


In [138]:
df_ibm_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [139]:
df_ibm_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [140]:
df_ibm_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-17,119.970001,119.970001,120.559998,119.019997,119.269997,3959100,NaN,NaN,120.522736,120.607473,122.894434,123.387170,117.658302,127.337892,113.877054,129.285204,116.503663
2021-02-18,120.730003,120.730003,120.940002,119.699997,120.500000,5398100,NaN,NaN,120.840816,120.225739,122.798580,122.801605,118.880026,125.884348,114.567129,129.172198,116.424963
2021-02-19,118.989998,118.989998,120.760002,118.379997,120.750000,6578700,NaN,NaN,120.797401,119.679540,122.717268,122.910881,118.683921,123.044776,116.314304,129.180605,116.253931
2021-02-22,120.860001,120.860001,121.129997,118.440002,118.500000,5879200,NaN,NaN,120.865001,119.870243,122.653422,122.934059,118.795943,123.033291,116.707195,129.125993,116.180851
2021-02-23,120.709999,120.709999,121.349998,119.800003,120.919998,4531300,NaN,NaN,120.738000,120.054926,122.565260,122.653059,118.822942,122.935035,117.174818,129.020870,116.109651


#### Apple Inc. Stock train test split

In [141]:
df_aapl = df_aapl.loc[data_start:].copy()

In [142]:
df_aapl_train, df_aapl_test, df_aapl_valid = train_test_valid_split(df_aapl, prediction_length=prediction_length[1])
print(len(df_aapl), len(df_aapl_train), len(df_aapl_test), len(df_aapl_valid))

4172 4132 4152 20


In [143]:
df_aapl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [144]:
df_aapl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [145]:
df_aapl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-17,130.839996,130.839996,132.220001,129.470001,131.250000,97918500.0,NaN,NaN,135.052515,136.138230,131.424077,138.995849,131.109180,142.966393,129.310068,142.185129,120.663025
2021-02-18,129.710007,129.710007,130.000000,127.410004,129.199997,96856700.0,NaN,NaN,134.649500,136.032081,131.576925,139.821352,129.477647,143.199620,128.864541,142.007567,121.146283
2021-02-19,129.869995,129.869995,130.710007,128.800003,130.240005,87668800.0,NaN,NaN,133.917999,135.692292,131.703018,139.545200,128.290798,143.360206,128.024378,141.887890,121.518145
2021-02-22,126.000000,126.000000,129.720001,125.599998,128.009995,103916400.0,NaN,NaN,132.842000,135.049167,131.739129,139.969011,125.714989,143.693038,126.405296,141.827686,121.650572
2021-02-23,125.860001,125.860001,126.709999,118.389999,123.760002,158273000.0,NaN,NaN,131.737000,134.206829,131.824364,139.462184,124.011815,142.990963,125.422695,141.631303,122.017424


#### Amazon Stock train test split

In [146]:
df_amzn = df_amzn.loc[data_start:].copy()

In [147]:
df_amzn_train, df_amzn_test, df_amzn_valid = train_test_valid_split(df_amzn, prediction_length=prediction_length[1])
print(len(df_amzn), len(df_amzn_train), len(df_amzn_test), len(df_amzn_valid))

4172 4132 4152 20


In [148]:
df_amzn_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [149]:
df_amzn_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [150]:
df_amzn_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-17,3308.639893,3308.639893,3320.909912,3259.500000,3263.600098,3297500.0,NaN,NaN,3302.762964,3295.481970,3225.831777,3360.352910,3245.173017,3381.043039,3209.920902,3380.877493,3070.786062
2021-02-18,3328.229980,3328.229980,3338.000000,3273.939941,3282.419922,3027400.0,NaN,NaN,3304.332959,3298.724475,3229.144775,3363.927436,3244.738482,3384.078129,3213.370821,3385.745319,3072.544232
2021-02-19,3249.899902,3249.899902,3333.500000,3245.750000,3328.229980,4305200.0,NaN,NaN,3296.222949,3295.869971,3230.982773,3361.491566,3230.954332,3383.838238,3207.901704,3386.327284,3075.638263
2021-02-22,3180.739990,3180.739990,3232.320068,3172.260010,3208.129883,3515700.0,NaN,NaN,3279.081958,3290.295471,3231.051772,3365.634028,3192.529888,3392.252777,3188.338165,3386.301978,3075.801567
2021-02-23,3194.500000,3194.500000,3204.729980,3093.600098,3127.030029,4677200.0,NaN,NaN,3266.237964,3285.320471,3232.857773,3361.541549,3170.934379,3395.865222,3174.775720,3384.134985,3081.580562


#### Alphabet Inc. Stock train test split

In [151]:
df_googl = df_googl.loc[data_start:].copy()

In [152]:
df_googl_train, df_googl_test, df_googl_valid = train_test_valid_split(df_googl, prediction_length=prediction_length[1])
print(len(df_googl), len(df_googl_train), len(df_googl_test), len(df_googl_valid))

4172 4132 4152 20


In [153]:
df_googl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [154]:
df_googl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [155]:
df_googl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-17,2118.620117,2118.620117,2122.439941,2088.350098,2093.979980,1014700.0,NaN,NaN,2086.082983,1981.576483,1846.484194,2126.467383,2045.698584,2202.624080,1760.528886,2112.159135,1580.809254
2021-02-18,2105.810059,2105.810059,2121.989990,2093.330078,2105.000000,1127300.0,NaN,NaN,2090.776001,1992.863489,1852.125195,2127.885024,2053.666978,2215.137461,1770.589516,2127.626424,1576.623967
2021-02-19,2088.810059,2088.810059,2118.580078,2083.429932,2107.790039,1896900.0,NaN,NaN,2094.294019,2003.096490,1857.560796,2120.949812,2067.638226,2223.129983,1783.062998,2140.850093,1574.271498
2021-02-22,2054.260010,2054.260010,2080.330078,2048.000000,2053.600098,1473300.0,NaN,NaN,2090.837012,2011.181488,1862.419397,2127.667275,2054.006748,2225.933273,1796.429703,2150.760197,1574.078597
2021-02-23,2060.120117,2060.120117,2072.500000,1990.229980,2013.989990,1732900.0,NaN,NaN,2088.397021,2019.473492,1868.064600,2130.009741,2046.784302,2227.934498,1811.012487,2160.668947,1575.460252


### Metrics computation

Computing metrics on the benchmark model will give me a good reference to evaluate the deep learning model after training.

As can be presumed and seen from data prints, first `n` values of validation data are `NaN` so, it will be better just to exclude the first `n-1` values from the validation set to avoid misleading values to be catch in metrics evaluation. 

In [156]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

#### IBM stock

In [157]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [158]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [159]:
#print(ibm_ma_mse_loss)

In [160]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [161]:
ibm_ma_mae_loss = mean_absolute_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [162]:
print(ibm_ma_mae_loss)

2.454520511627197


Root Mean Squared Error

In [163]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str], squared=False)

In [164]:
print(ibm_ma_mse_loss)

3.1869954720383604


Mean Absolute Percentage Error

In [165]:
ibm_ma_map_loss = mean_absolute_percentage_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [166]:
print(ibm_ma_map_loss)

0.01955582537392973


R<sup>2</sup> score

In [167]:
ibm_ma_r2_score = r2_score(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [168]:
print(ibm_ma_r2_score)

0.0017912993352110496


In [169]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [170]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [171]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [172]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [173]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [174]:
#print(aapl_ma_mse_loss)

In [175]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [176]:
aapl_ma_mae_loss = mean_absolute_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [177]:
print(aapl_ma_mae_loss)

6.557271537780761


##### Root Mean Squared Error

In [178]:
aapl_ma_rmse_loss = mean_squared_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str], squared=False)

In [179]:
print(aapl_ma_rmse_loss)

7.194991344769908


##### Mean Absolute Percentage Error

In [180]:
aapl_ma_map_loss = mean_absolute_percentage_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [181]:
print(aapl_ma_map_loss)

0.053283463604940996


R<sup>2</sup> score

In [182]:
aapl_ma_r2_score = r2_score(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [183]:
print(aapl_ma_r2_score)

-2.7175937372640955


In [184]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [185]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [186]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [187]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [188]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [189]:
#print(amzn_ma_mse_loss)

In [190]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [191]:
amzn_ma_mae_loss = mean_absolute_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [192]:
print(amzn_ma_mae_loss)

111.63395568847656


Root Mean Squared Error

In [193]:
amzn_ma_rmse_loss = mean_squared_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str], squared=False)

In [194]:
print(amzn_ma_rmse_loss)

133.609757631983


##### Mean Absolute Percentage Error

In [195]:
amzn_ma_map_loss = mean_absolute_percentage_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [196]:
print(amzn_ma_map_loss)

0.0364328723766882


R<sup>2</sup> score

In [197]:
amzn_ma_r2_score = r2_score(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [198]:
print(amzn_ma_r2_score)

-0.8045289277163996


In [199]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [200]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [201]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [202]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [203]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [204]:
#print(googl_ma_mse_loss)

In [205]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [206]:
googl_ma_mae_loss = mean_absolute_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [207]:
print(googl_ma_mae_loss)

23.95187988281259


Root Mean Squared Error

In [208]:
googl_ma_rmse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str], squared=False)

In [209]:
print(googl_ma_rmse_loss)

23.95187988281259


##### Mean Absolute Percentage Error

In [210]:
googl_ma_map_loss = mean_absolute_percentage_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [211]:
print(googl_ma_map_loss)

0.020577607487795004


R<sup>2</sup> score

In [212]:
googl_ma_r2_score = r2_score(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [213]:
print(googl_ma_r2_score)

-1.8450668340829974


In [214]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [215]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [216]:
#print(googl_ma_mse_loss)

Volatility

In [217]:
print(volatility(df_ibm_valid['Adj Close'], n))

0.5355351004479918


In [218]:
print(volatility(df_aapl_valid['Adj Close'], n))

0.7329005082594083


In [219]:
print(volatility(df_amzn_valid['Adj Close'], n))

520.6656214083532


In [220]:
print(volatility(df_googl_valid['Adj Close'], n))

55.100503589887225


As expected, loss augments as we observe moving average on larger windows.
Also, we can observe that Amazon.com and Alphabet Inc. have greater losses, that also corresponds to higher volatility.